### Imports and Loading Data

In [1]:
# imports
import psycopg2
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import config as c

In [2]:
# connection to database
connection = psycopg2.connect(
    host = c.host,
    port = c.port,
    user = c.user,
    password = c.password,
    database = c.database
    )
cursor=connection.cursor()

In [3]:
# SQL query
data_sql = """
SELECT *
FROM final_dataset;
"""

# load in tables as dataframes
data_df = pd.read_sql(data_sql, con=connection)

### Setup Confusion Matrix

In [6]:
# define conditions
conditions = [
  (data_df['HallOfFameStatus'] == 'Not Inducted') & (data_df['ML_Prediction'] == 0),
  (data_df['HallOfFameStatus'] == 'Hall of Fame Member') & (data_df['ML_Prediction'] == 0),
  (data_df['HallOfFameStatus'] == 'Not Inducted') & (data_df['ML_Prediction'] == 1),
  (data_df['HallOfFameStatus'] == 'Hall of Fame Member') & (data_df['ML_Prediction'] == 1)
]
outputs = ['True Negative', 'False Negative', 'False Positive', 'True Positive']

# add ConfusionMatrix column to df
data_df['ConfusionMatrix'] = pd.Series(np.select(conditions, outputs, 'Error'))

# confirm added column
data_df.head()

,index,GUID,Name,HallOfFameStatus,HallofFameClass,YearDrafted,TO_YEAR,Years_Played,HOF_Elgibility_Year,Pick,...,EFF,idPlayer,numberRound,BIRTHDATE,POSITION,AGE_ROOKIE_SEASON,Draft_Decade,numBallotsBeforeInduct,ML_Prediction,ConfusionMatrix
0,0,MITCHMCGARY2014,Mitch McGary,Not Inducted,NaN,2014,2015,2,2020,21,...,9.0,203956.0,1,1992-06-06,Forward,22.36,2010s,NaN,0,True Negative
1,1,PJHAIRSTON2014,PJ Hairston,Not Inducted,NaN,2014,2015,2,2020,26,...,4.3,203798.0,1,1992-12-24,Forward,21.81,2010s,NaN,0,True Negative
2,2,KJMCDANIELS2014,KJ McDaniels,Not Inducted,NaN,2014,2016,3,2021,32,...,7.6,203909.0,2,1993-02-09,Guard,21.68,2010s,NaN,0,True Negative
3,3,CLEANTHONYEARLY2014,Cleanthony Early,Not Inducted,NaN,2014,2015,2,2020,34,...,4.8,203921.0,2,1991-04-17,Forward,23.50,2010s,NaN,0,True Negative
4,4,CORYJEFFERSON2014,Cory Jefferson,Not Inducted,NaN,2014,2015,2,2020,60,...,4.8,203928.0,2,1990-12-26,Forward,23.80,2010s,NaN,0,True Negative


### Append Database

In [9]:
# connection to database
connection = psycopg2.connect(
    host = c.host,
    port = c.port,
    user = c.user,
    password = c.password,
    database = c.database
    )
cursor=connection.cursor()

# define which rows to use
rows = zip(data_df.GUID, data_df.ConfusionMatrix)

# SQL: temp table
cursor.execute("""CREATE TEMP TABLE "bdb_Draft_test" ("GUID" text, "ConfusionMatrix" text) ON COMMIT DROP""")
cursor.executemany("""INSERT INTO "bdb_Draft_test" ("GUID", "ConfusionMatrix") VALUES(%s, %s)""", rows)

# SQL: add and append
cursor.execute("""
    ALTER TABLE final_dataset
    DROP COLUMN IF EXISTS "ConfusionMatrix";
    
    ALTER TABLE final_dataset
    ADD COLUMN "ConfusionMatrix" text;
   
    UPDATE final_dataset
    SET "ConfusionMatrix" = "bdb_Draft_test"."ConfusionMatrix"
    FROM "bdb_Draft_test"
    WHERE final_dataset."GUID" = "bdb_Draft_test"."GUID";
    """)

# commit and close 
cursor.rowcount
connection.commit()
cursor.close()
connection.close()